In [3]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect('../database/pythonsqlite.db')
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)

    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query, (relayNumber,))
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)


        cursor.close()

        return result_list
    else:
        cursor.close()
        return result_list


def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        df = pd.read_sql_query(f"SELECT start_timestamp, end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE start_timestamp BETWEEN '{start_of_day_str}' AND '{end_of_day_str}'", conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        print(df)

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()


def calculateAutoModeValue(relayNumber):
    db = DatabaseManager()

    if(relayNumber == 1):
        relayPower = db.read_relaysettings_table()[0][2] # relay 1 power
    elif(relayNumber == 2):
        relayPower = db.read_relaysettings_table()[0][3] # relay 2 power
    
    last_known_times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    last_known_times_toturn_on = int(last_known_times_toturn_on[0][0]) * int(relayPower)
    last_24hrs_usage = db.read_datacache_last_24hrs_consumption()[0][0] # A
    
    """
    Formula to calculate auto_mode value
    A = Last 24hrs consumption
    B = Power needed for Relay1 or Relay 2
    Result = A/B

    Example: A = 20KW; B = 5KW
        Result = 20/5 
                = 4 times (this we have to turn on and turn off the relay automatically)

    Escalation Example:
        Lets say times_turnon=6*Relay power (20kwh)
            E old = 6*20 = 120 kwh 
        If the last 24hrs consumption is greater or equal to (E Old) then multiply with escalation value
    """
    
    # Logic for Escalation --> 1.3 default value
    # Higher the demand higher the turn on time
    if(last_known_times_toturn_on >= last_24hrs_usage):
        no_of_times_to_activate_automode = last_24hrs_usage * 1.3 / int(relayPower)
        db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
    else:
        no_of_times_to_activate_automode = int(last_24hrs_usage) / int(relayPower)
        db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 

    return True



def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        query = f"SELECT REPLACE(start_timestamp, 'T', ' ') AS start_timestamp, REPLACE(end_timestamp, 'T', ' ') AS end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE (start_timestamp >= '{start_of_day_str}' AND start_timestamp <= '{end_of_day_str}') OR (end_of_day_str >= '{start_of_day_str}' AND end_of_day_str <= '{end_of_day_str}')"

        df = pd.read_sql_query(query, conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()



#getAutoModeStatus()

def read_automode_24hrs_before(relayNumber):
    conn = sqlite3.connect('../database/pythonsqlite.db')
    cursor = conn.cursor()
    
    current_datetime = datetime.now() - timedelta(hours=24)
    start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

    query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
    cursor.execute(query, (relayNumber,))
    
    rows = cursor.fetchall()
    result_list = []
    for row in rows:
        result_list.append(row)


    if int(result_list[0][0]) == 0: # if no results then check 48 hrs before data

        current_datetime = datetime.now() - timedelta(hours=48)
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)

        query = f"SELECT COALESCE(SUM(times_to_turnon), 0) AS result FROM automode WHERE relaynumber=? AND status=1 AND datetime BETWEEN '{start_of_day}' AND '{end_of_day}' ORDER BY id DESC LIMIT 1;"
        cursor.execute(query)
        
        rows = cursor.fetchall()
        result_list = []
        for row in rows:
            result_list.append(row)

        cursor.close()
        return result_list
    else:
        cursor.close()
        return result_list


def auto_mode(relayNumber):
    db = DatabaseManager()
    times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    if not times_toturn_on:
        return
    else:
        times_toturn_on = int(times_toturn_on[0][0])
    
    print("No of times to turn on the relay", times_toturn_on)
    future_df = AwattarServices().AWATTAR_FUTURE_PRICE()
    print("Data from BEFORE modification", future_df)

    date_time = datetime.now()
    print(date_time)

    # delete past values from future_df
    # self.future_df = self.future_df[self.future_df['start_timestamp'] >= self.date_time.now()]

    # find min 2 values of future_df
    future_df.sort_values(by=['marketprice'], inplace=True)
    future_df = future_df.iloc[:times_toturn_on]
    future_df.sort_values(by=['start_timestamp'], inplace=True)
    # add extra columns to data frame and to match database table "automaterelay"
    future_df['triggerstatus'] = True
    future_df['unit'] = "kWh"
    future_df['relaynumber'] = relayNumber

    print("Data from AFTER modification", future_df)

    if len(future_df) > 0:
        print("Found matching auto mode for relay:", relayNumber)
        #conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
        #self.future_df.to_sql('automaterelay', conn, index=False, if_exists='append') # replace the dataset
        #self.future_df.to_sql('automaterelay_report', conn, index=False, if_exists='append') # for report purpose
        #conn.close()
    else:
        print("No matching auto mode")

    #self.turn_on_turn_off(relayNumber)



auto_mode(1)


Current working directory: /home/pi/smart_plug/python_files
init called
2.6.0
No of times to turn on the relay 7
Data from BEFORE modification        start_timestamp       end_timestamp  marketprice     unit
0  2023-12-06 00:00:00 2023-12-06 01:00:00        94.97  Eur/MWh
1  2023-12-06 01:00:00 2023-12-06 02:00:00        93.10  Eur/MWh
2  2023-12-06 02:00:00 2023-12-06 03:00:00        92.57  Eur/MWh
3  2023-12-06 03:00:00 2023-12-06 04:00:00        90.96  Eur/MWh
4  2023-12-06 04:00:00 2023-12-06 05:00:00        90.59  Eur/MWh
5  2023-12-06 05:00:00 2023-12-06 06:00:00        93.89  Eur/MWh
6  2023-12-06 06:00:00 2023-12-06 07:00:00       108.20  Eur/MWh
7  2023-12-06 07:00:00 2023-12-06 08:00:00       133.78  Eur/MWh
8  2023-12-06 08:00:00 2023-12-06 09:00:00       167.00  Eur/MWh
9  2023-12-06 09:00:00 2023-12-06 10:00:00       166.54  Eur/MWh
10 2023-12-06 10:00:00 2023-12-06 11:00:00       150.00  Eur/MWh
11 2023-12-06 11:00:00 2023-12-06 12:00:00       146.29  Eur/MWh
12 2023-12-0